# Final code

In [1]:
!pip install langchain_google_genai

In [2]:
!pip install python-dotenv

  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)


In [1]:
# імпорти
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from typing_extensions import Literal
import pandas as pd
import time
import os
from dotenv import load_dotenv

In [2]:
# Завантажуємо .env файл
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

In [3]:
# Налаштування моделі
model_name = "gemini-2.0-flash-lite"
llm = ChatGoogleGenerativeAI(
    model=model_name,
    api_key=api_key,
    temperature=0.8,
    max_retries=2,
    top_p=0.9,
    top_k=40,
)

In [4]:
# Ініціалізація моделі
class OutputModel(BaseModel):
    title: str = Field(description="Заголовок")
    text: str = Field(description="Текст новини (300-500 слів)")
    topic: str = Field(description="Обрана тема")
    type_news: Literal["Маніпуляція", "Дезінформація", "Пропаганда"]

parser = JsonOutputParser(pydantic_object=OutputModel)

In [5]:
# Повідомлення
system_message = """Ви — досвідчений журналіст із понад 10 роками роботи в провідних українських та міжнародних медіа.
Ваші тексти вирізняються глибиною аналізу, структурованістю та дотриманням високих стандартів журналістики.
Ви володієте майстерністю фактчекінгу, балансу думок і доступного викладу складних тем.
Ваш стиль — виважений, аргументований, орієнтований на критично мислячу аудиторію.
Кожен ваш матеріал формує довіру, викликає інтелектуальну зацікавленість і спонукає до осмислення подій.\
"""

human_message_template = """Згенеруй **фейкову, але правдоподібну** новину у стилі провідних українських медіа 2022–2025 років. \
Тематика — **політика**, **економіка**, **суспільство** або **військові події**.

---

### Тип новини (вибери один):
1. **Маніпуляція** — достовірні факти, подані у викривленому або контекстуально упередженому вигляді.
2. **Дезінформація** — повністю вигадана або перекручена подія, що не відповідала дійсності.
3. **Пропаганда** — наратив, який шкодить іміджу України, відповідає інтересам ворожої сторони (наприклад, РФ).

---

### Вимоги до тексту:
- **Почни з головного** — подія, її суть, причетні особи або інституції.
- Пиши **доступною, живою мовою** — без офіціозу, штампів чи технократичного жаргону.
- Стиль має бути **максимально реалістичним**, у форматі репортажу або аналітичної замітки.
- Включай **реальні імена посадовців, державних установ і фактів** (у межах 2022–2025 рр.).
- Додай **коментарі експертів або посадових осіб** — вигадані, але переконливі.
- **Не вигадуй географічних назв**, не використовуй очевидні кліше чи фантастичні події.
- Орієнтуйся на стиль **«Української правди», «bihus.info», «ТСН»** — стриманий, динамічний, фактологічний.

---

{format_of_output}
"""


In [6]:
# генерація новини
def generate_one_news():
    human_message = human_message_template.format(
        format_of_output=parser.get_format_instructions()
    )
    messages = [("system", system_message), ("human", human_message)]

    response = llm.invoke(messages)
    return parser.parse(response.content)

In [7]:
# додавання до датасету
def append_to_csv(news, path= "..\\..\\data\\fake_news_dataset.csv"):
    df = pd.DataFrame(
        [
            {
                "title": news['title'],
                "text": news['text'],
                "topic": news['topic'],
                "type": news['type_news']
            }
        ]
    )

    if not os.path.exists(path):
        df.to_csv(path, index=False, encoding="utf-8")
    else:
        df.to_csv(path, mode="a", index=False, header=False, encoding="utf-8")

In [8]:
n = 5
delay = 2

for i in range(n):
    news = generate_one_news()
    print(f"[{i+1}/{n}] {news['title']}")
    append_to_csv(news)
    time.sleep(delay)

print(f"\n Згенеровано {n} новин. Результати збережено.")

[1/5] ЗМІ: Європейські партнери висловлюють стурбованість темпами реформ в Україні напередодні зими
[2/5] ЗМІ: Відставка Резнікова стала результатом незадоволення західних партнерів темпами реформ в оборонній сфері
[3/5] Злив даних з Офісу Президента: чия вина та які наслідки для міжнародної підтримки?
[4/5] Зростання експорту зерна через Польщу: чи не ховається за показниками тіньовий бізнес?
[5/5] Зростання імпорту російського газу через посередників: нові схеми чи стара практика?

 Згенеровано 5 новин. Результати збережено.


In [9]:
import pandas as pd

df = pd.read_csv("..\\..\\data\\fake_news_dataset.csv")
df

,title,text,topic,type
0,ЗМІ: Європейські партнери висловлюють стурбова...,"Київ, Україна – Останніми тижнями зростає зане...",Політика,Маніпуляція
1,ЗМІ: Відставка Резнікова стала результатом нез...,"Київ, Україна – За інформацією, отриманою з кі...",Політика,Маніпуляція
2,Злив даних з Офісу Президента: чия вина та які...,Несподіваний витік внутрішньої документації з ...,Політика,Маніпуляція
3,Зростання експорту зерна через Польщу: чи не х...,Київ – Останні місяці демонструють стрімке зро...,Економіка,Маніпуляція
4,Зростання імпорту російського газу через посер...,"Київ, Україна – Останні місяці спостерігається...",Економіка,Маніпуляція


In [11]:
print(df['text'][0])

Київ, Україна – Останніми тижнями зростає занепокоєння серед західних партнерів України щодо прогресу в проведенні ключових реформ, зокрема у сфері боротьби з корупцією та прозорості державних закупівель. Джерела в дипломатичних колах ЄС та США повідомляють про низку неформальних зустрічей, на яких обговорювалася ситуація з виконанням Україною рекомендацій Європейської комісії щодо членства в ЄС.

Особливу увагу привертає затримка з призначенням нового керівника Національного антикорупційного бюро України (НАБУ) та невирішеність питання щодо реформування судової системи. "Ми бачимо позитивні зрушення, але темп реформ не відповідає критично важливим викликам, з якими Україна стикається, зокрема, напередодні зими", - цитує джерело в Брюсселі анонімного представника Європейської комісії.

Ситуацію ускладнює також питання прозорості використання фінансової допомоги, що надходить від міжнародних партнерів. Незважаючи на офіційні заяви української влади про ефективний контроль за розподілом 